In [1]:
!python -m pip install obsws-python


In [2]:
import time
from threading import Thread, Barrier
import obsws_python as obs

TARGETS = [ 
    # IP address, the websocket port, obs websocket password, name for printing messages
    {"host": "10.0.0.137", "port": 4455, "password": "GARN_air", "name": "AIR COMPUTER"},
    # {"host": "192.168.0.141", "port": 4455, "password": "GARN_air", "name": "AIR COMPUTER"},
    # {"host": "192.168.0.142", "port": 4455, "password": "GARN_ground", "name": "GROUND COMPUTER"},
]

def start_record_job(tgt, go_barrier):
    # Requests client connection to OBS
    cl = obs.ReqClient(host=tgt["host"], port=tgt["port"], password=tgt["password"], timeout=3)
    
    # Asks OBS whether recording is already active
    try:
        status = cl.get_record_status()
        if getattr(status, "output_active", False):
            print(f'[{tgt["name"]}] Already recording.')
            return
        
        # The following ensures all OBS recordings start simultaneously
        go_barrier.wait()
        cl.start_record()
        
        # Wait for OBS to let OBS update its state
        time.sleep(0.2)
        
        # Asks again for status
        confirm = cl.get_record_status()
        if getattr(confirm, "output_active", False):
            print(f'[{tgt["name"]}] Recording started.')
        else:
            print(f'[{tgt["name"]}] Failed to start recording.')
            
    # error prints if anything goes wrong
    except Exception as e:
        print(f'[{tgt["name"]}] ERROR: {e}')

def main():
    # Early exit if no OBS instances
    if not TARGETS:
        print("No targets configured.")
        return
    
    # Threads wait for all OBS instances on the network
    barrier = Barrier(len(TARGETS))
    
    # Creates on thread per OBS computer & starts recording
    threads = [Thread(target=start_record_job, args=(t, barrier), daemon=True) for t in TARGETS]
    
    # Start all threads
    for th in threads: th.start()
    for th in threads: th.join()

main()

[AIR COMPUTER] Recording started.
